In [1]:
#Draft
import numpy as np
import igraph as ig
import os
import cv2 as cv
import base
import process_image
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import error
import time
import functools
import gsd.hoomd
from skimage.morphology import skeletonize_3d
import root

class Graph(object):
    """Generic SGT graph class: a specialised case of igraph Graph object with additional attributes
    defining geometric features, associated images, dimensionality etc
    
    Initialised from directory containing raw image data
    self._2d determined from the number of images with identical dimensions (suggesting a stack when > 1)
    
    Image shrinking/cropping is carried out at the gsd stage in analysis.
    I.e. full images are binarized but cropping their graphs may come after
    """
    def __init__(self, directory, child_dir='/Binarized'):
        if not isinstance(directory, str):
            raise TypeError
            
        self.dir = directory
        self.child_dir = child_dir
        self.stack_dir = self.dir + self.child_dir
        
        shape = []
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            shape.append(cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE).shape)
        
        if len(set(shape)) == len(shape):
            self._2d = True
            self.dim = 2
        else:
            self._2d = False
            self.dim = 3
        
    def binarize(self, options_dict=None):
        """Binarizes stack of experimental images using a set of image processing parameters in options_dict.
        Note this enforces that all images have the same shape as the first image encountered by the for loop.
        (i.e. the first alphanumeric titled image file)
        """
        
        if options_dict is None:
            options = self.dir + '/img_options.json'
            with open(options) as f:
                options_dict = json.load(f)

        if not os.path.isdir(self.dir + self.child_dir):
            os.mkdir(self.dir + self.child_dir)

        i=0
        for name in sorted(os.listdir(self.dir)):
            if not base.Q_img(name):
                continue
            else:
                img_exp = cv.imread(self.dir+'/'+name,cv.IMREAD_GRAYSCALE)
                if i == 0: shape = img_exp.shape
                elif img_exp.shape != shape: continue
                _, img_bin, _ = process_image.binarize(img_exp, options_dict)
                plt.imsave(self.dir+self.child_dir+'/slice'+str(i)+'.tiff', img_bin, cmap=cm.gray)
                i+=1
        
    def stack_to_gsd(self, name='skel.gsd', crop=None, skeleton=True, rotate=None, debubble=False):
        """Writes a .gsd file from the object's directory.
        The name of the written .gsd is set as an attribute so it may be easily matched with its Graph object 
        Running this also sets the positions, shape attributes
        
        """
        
        start = time.time()
        self.gsd_name = self.dir + '/' + name
        img_bin=[]
        
        #Initilise i such that it starts at the lowest number belonging to the images in the stack_dir
        #First require boolean mask to filter out non image files
        olist = np.asarray(sorted(os.listdir(self.stack_dir)))
        mask = list(base.Q_img(olist[i]) for i in range(len(olist)))
        if len(mask) == 0:
            raise error.ImageDirectoryError(self.stack_dir)
        fname = sorted(olist[mask])[0] #First name
        i = int(os.path.splitext(fname)[0][5:]) #Strip file type and 'slice' then convert to int
        
        #Generate 3d (or 2d) array from stack
        for fname in sorted(os.listdir(self.stack_dir)):
            if base.Q_img(fname):
                img_slice = cv.imread(self.stack_dir+'/slice'+str(i)+'.tiff',cv.IMREAD_GRAYSCALE)
                if rotate:
                    image_center = tuple(np.array(img_slice.shape[1::-1]) / 2)
                    rot_mat = cv.getRotationMatrix2D(image_center, rotate, 1.0)
                    img_slice = cv.warpAffine(img_slice, rot_mat, img_slice.shape[1::-1], flags=cv.INTER_LINEAR)
                img_bin.append(img_slice)
                i=i+1
            else:
                continue
                
        img_bin = np.asarray(img_bin)
        #Roll axes such that img_bin.shape[2]==1 when the graph is 2D
        img_bin = np.swapaxes(img_bin, 0, 2)

        #Note that numpy array slicing operations are carried out in reverse order!
        #(...hence crop 2 and 3 before 0 and 1)
        if crop and self._2d:
            img_bin = img_bin[crop[2]:crop[3], crop[0]:crop[1]]
        elif crop:
            #TODO figure tf this bit out
            img_bin = img_bin[crop[0]:crop[1], crop[2]:crop[3], crop[4]:crop[5]]
        
        if skeleton:
            img_bin = skeletonize_3d(np.asarray(img_bin))
        else:
            img_bin = np.asarray(img_bin)
        
        self.img_bin_3d = img_bin            #Always 3d, even for 2d images
        self.img_bin = np.squeeze(img_bin)   #3d for 3d images, 2d otherwise
        
        positions = np.asarray(np.where(np.asarray(img_bin) != 0)).T
        self.shape = np.asarray(list(max(positions.T[i])+1 for i in (0,1,2)[0:self.dim]))
        self.positions = positions
        
        with gsd.hoomd.open(name=self.gsd_name, mode='wb') as f:
            s = gsd.hoomd.Snapshot()
            s.particles.N = len(positions)
            s.particles.position = base.shift(positions)
            s.particles.types = ['A']
            s.particles.typeid = ['0']*s.particles.N
            f.append(s)
        
        end = time.time()
        print('Ran stack_to_gsd() in ', end-start, 'for gsd with ', len(positions), 'particles')
        
        if debubble:
            base.debubble(self)
            self.gsd_name = self.dir + '/debubbled_' + name
            
            
    def G_u(self):
        """Sets unweighted igraph object as an attribute
        """
        G =  base.gsd_to_G(self.gsd_name, _2d = self._2d)
        self.Gr = G
        
    def weighted_Laplacian(self):

        L=np.asarray(self.Gr.laplacian(weights='weight'))
        self.L = L

class ResistiveNetwork(Graph):
    """Child of generic SGT Graph class.
    Equipped with methods for analysing resistive flow networks

    """
    def __init__(self, directory):
        super().__init__(directory)
        
    def potential_distribution(self, plane, boundary1, boundary2, R_j=0, rho_dim=1, F_dim=1):
        """Solves for the potential distribution in a weighted network.
        Source and sink nodes are connected according to a penetration boundary condition.
        Sets the corresponding weighted Laplacian, potential and flow attributes.
        The 'plane' arguement defines the axis along which the boundary arguements refer to.
        R_j='infinity' enables the unusual case of all edges having the same unit resistance.
        """
        
        self.G_u()
        print('pre sub has ', self.Gr.vcount(), ' nodes')
        self.Gr = base.sub_G(self.Gr)
        print('post sub has ', self.Gr.vcount(), ' nodes')
        if R_j != 'infinity':
            print(self.Gr.vcount())
            self.Gr = base.add_weights(self, weight_type='Resistance', R_j=R_j, rho_dim=rho_dim)
            print(self.Gr.vcount())
            weight_array = np.asarray(self.Gr.es['weight']).astype(float)
            weight_array = weight_array[~np.isnan(weight_array)]
            weight_avg =np.mean(weight_array)
        else:
            self.Gr.es['weight'] = np.ones(self.Gr.ecount())
            weight_avg = 1

        #Add source and sink nodes:
        source_id = max(self.Gr.vs).index + 1
        sink_id = source_id + 1
        self.Gr.add_vertices(2)

        print('Graph has max ', self.shape)
        axes = np.array([0,1,2])[0:self.dim]
        indices = axes[axes!=plane]
        plane_centre1 = np.zeros(self.dim, dtype=int)
        delta = np.zeros(self.dim, dtype=int)
        delta[plane] = 10 #Arbitrary. Standardize?
        for i in indices: plane_centre1[i] = self.shape[i]/2
        plane_centre2 = np.copy(plane_centre1)
        plane_centre2[plane] = self.shape[plane]
        source_coord = plane_centre1 - delta 
        sink_coord = plane_centre2 + delta
        print('source coord is ', source_coord)
        print('sink coord is ', sink_coord)
        self.Gr.vs[source_id]['o'] = source_coord
        self.Gr.vs[sink_id]['o'] = sink_coord

    #Connect nodes on a given boundary to the external current nodes
        print('Before connecting external nodes, G has vcount ', self.Gr.vcount())
        for node in self.Gr.vs:
            if node['o'][plane] > boundary1[0] and node['o'][plane] < boundary1[1]:
                self.Gr.add_edges([(node.index, source_id)])
                self.Gr.es[self.Gr.get_eid(node.index,source_id)]['weight'] = weight_avg
                self.Gr.es[self.Gr.get_eid(node.index,source_id)]['pts'] = base.connector(source_coord,node['o'])
            if node['o'][plane] > boundary2[0] and node['o'][plane] < boundary2[1]:
                self.Gr.add_edges([(node.index, sink_id)])
                self.Gr.es[self.Gr.get_eid(node.index,sink_id)]['weight'] = weight_avg 
                self.Gr.es[self.Gr.get_eid(node.index,sink_id)]['pts'] = base.connector(sink_coord,node['o'])

    #Write skeleton connected to external node
        print(self.Gr.is_connected(), ' connected')
        print('After connecting external nodes, G has vcount ', self.Gr.vcount())
        connected_name = os.path.split(self.gsd_name)[0] + '/connected_' + os.path.split(self.gsd_name)[1] 
        #connected_name = self.stack_dir + '/connected_' + self.gsd_name 
        base.G_to_gsd(self.Gr, connected_name)

        self.weighted_Laplacian()
        F = np.zeros(sink_id+1)
        print(F.shape,'F')
        print(self.L.shape, 'L')
        F[source_id] = F_dim
        F[sink_id] = -F_dim
        np.save(self.stack_dir+'/L.npy',self.L)
        np.save(self.stack_dir+'/F.npy',F)
        P = np.matmul(np.linalg.pinv(self.L, hermitian=True),F)
        np.save(self.stack_dir+'/P.npy',P)

        self.P = P
        self.F = F

class StructuralNetwork(Graph):
    """Child of generic SGT Graph class.
    Equipped with methods for analysing structural networks
    """
    def __init__(self, directory):
        super().__init__(directory)
        
    def G_calc(self):
        avg_indices = dict()

        operations = [self.Gr.diameter, self.Gr.density, self.Gr.transitivity_undirected, self.Gr.assortativity_degree]
        names = ['Diameter', 'Density', 'Clustering', 'Assortativity by degree']

        for operation,name in zip(operations,names):
            start = time.time()
            avg_indices[name] = operation()
            end = time.time()
            print('Calculated ', name, ' in ', end-start)
            
        self.G_attributes = avg_indices
        
    def node_calc(self):
        self.Betweenness = self.Gr.betweenness()
        self.Closeness = self.Gr.closeness()
        self.Degree = self.Gr.degree()

In [2]:
g = ResistiveNetwork('TestData/pores-hi-res-crop')
g.stack_to_gsd(crop=[0,100,0,100,0,100])
g.G_u()
g.potential_distribution(0, [0,20], [80,100])
base.Node_labelling(g, g.P, 'P', 'test1.gsd')


h = ResistiveNetwork('TestData/pores-hi-res-crop')
h.stack_to_gsd(crop=[100,300,100,300,100,300])
h.G_u()
h.potential_distribution(0, [0,20], [180,200])
base.Node_labelling(h, h.P, 'P', 'test2.gsd')

Ran stack_to_gsd() in  26.416732788085938 for gsd with  43095 particles
gsd_to_G canvas has shape  (100, 100, 100)
[[ 0  1 10]]
(0, 227, array([[ 0,  1, 10],
       [ 1,  1, 10],
       [ 2,  1, 10],
       [ 3,  1, 10],
       [ 4,  1, 10],
       [ 5,  1, 10],
       [ 6,  1, 10],
       [ 7,  1, 10],
       [ 8,  2,  9],
       [ 9,  3,  9],
       [10,  4,  9]], dtype=int16))
Ran gsd_to_G in  3.744230031967163 for a graph with  2015 nodes.
gsd_to_G canvas has shape  (100, 100, 100)
[[ 0  1 10]]
(0, 227, array([[ 0,  1, 10],
       [ 1,  1, 10],
       [ 2,  1, 10],
       [ 3,  1, 10],
       [ 4,  1, 10],
       [ 5,  1, 10],
       [ 6,  1, 10],
       [ 7,  1, 10],
       [ 8,  2,  9],
       [ 9,  3,  9],
       [10,  4,  9]], dtype=int16))
Ran gsd_to_G in  3.6526620388031006 for a graph with  2015 nodes.
pre sub has  2015  nodes
pre sub has  2015  nodes
post sub has  1600  nodes
post sub has  1600  nodes
1600
graph_shape is  [99, 99]  and img_shape is  (1024, 1536)
Loaded img 

/Users/alaink/Software/StructuralGTEdits/StructuralGT/GetWeights_3d.py:32: RuntimeWarning: invalid value encountered in true_divide
  return vec/np.linalg.norm(vec)


Added weights to a graph  with  1600 nodes in  0.3894069194793701
1600
Graph has max  [100 100 100]
source coord is  [-10  50  50]
sink coord is  [110  50  50]
Before connecting external nodes, G has vcount  1602
True  connected
After connecting external nodes, G has vcount  1602
(1602,) F
(1602, 1602) L


implicit data copy when writing chunk: log/particles/P


Ran stack_to_gsd() in  28.616368055343628 for gsd with  138415 particles
gsd_to_G canvas has shape  (200, 200, 200)
[[ 0  0 21]
 [ 0  1 21]]
(1, 774, array([[0, 1, 8],
       [1, 2, 8],
       [2, 2, 8],
       [3, 2, 8],
       [4, 3, 8],
       [5, 3, 7],
       [6, 3, 7],
       [7, 3, 7],
       [8, 4, 7]], dtype=int16))
Ran gsd_to_G in  16.71473002433777 for a graph with  13217 nodes.
gsd_to_G canvas has shape  (200, 200, 200)
[[ 0  0 21]
 [ 0  1 21]]
(1, 774, array([[0, 1, 8],
       [1, 2, 8],
       [2, 2, 8],
       [3, 2, 8],
       [4, 3, 8],
       [5, 3, 7],
       [6, 3, 7],
       [7, 3, 7],
       [8, 4, 7]], dtype=int16))
Ran gsd_to_G in  16.649702310562134 for a graph with  13217 nodes.
pre sub has  13217  nodes
pre sub has  13217  nodes
post sub has  10028  nodes
post sub has  10028  nodes
10028
graph_shape is  [199, 199]  and img_shape is  (1024, 1536)
Loaded img in  0.0
img_bin has shape  (200, 200, 200)
Added weights to a graph  with  10028 nodes in  1.95744895935

implicit data copy when writing chunk: log/particles/P


In [3]:
g = StructuralNetwork('TestData/fibres-hi-res-crop')
g.stack_to_gsd(crop=[0,100,0,100,0,100])
g.G_u()
g.potential_distribution(0, [0,20], [80,100])

g.stack_to_gsd(crop=[100,300,100,300,100,300], name = 'skel2.gsd')
g.G_u()
g.potential_distribution(0, [0,20], [80,100])
base.Node_labelling(g, g.Degree, 'Degree', 'test2.gsd')



Ran stack_to_gsd() in  21.676934242248535 for gsd with  43395 particles
gsd_to_G canvas has shape  (100, 100, 100)
[[0 0 4]]
(0, 137, array([[0, 0, 4],
       [1, 0, 4],
       [2, 0, 4],
       [3, 0, 4],
       [4, 1, 4],
       [5, 1, 4],
       [6, 1, 4],
       [7, 1, 4],
       [8, 1, 4],
       [9, 0, 5]], dtype=int16))
Ran gsd_to_G in  3.7469260692596436 for a graph with  2083 nodes.


AttributeError: 'StructuralNetwork' object has no attribute 'potential_distribution'

In [ ]:
#Rotation test
#Continuous rotation analysis
#Testing base.py implementation
#This implementation rotates the image itself

from PIL import Image

g = ResistiveNetwork('TestData/AgNWN_10um')
g.binarize()

#thetas = np.linspace(0.01,350, 36)
thetas = (0,30,60,90,180)

#crop = 359

#Need to reassign weights from a temporary rotated binary image
#First write the temporary image
#Must calculate positions for the crop because this crop is origin cornered (and not origin centred, like the one above)
img_bin = cv.imread(g.stack_dir+'/slice0.tiff') #Original image
image_center = tuple(np.array(img_bin.shape[1::-1]) / 2)
short_length = img_bin.shape[img_bin.shape == max(img_bin.shape)]
long_length = max(img_bin.shape)
ISS = (short_length**2/2)**0.5
L1 = int((long_length - ISS)/2)
L3 = int(ISS+L1)
L2 = int((short_length - ISS)/2)
L4 = int(ISS+L2)
o_corn_crop = [L2,L1,L4,L3]
dims = L2-L1


R_j = 0
O_eff_df = []
Ax_df = []
Ay_df = []
theta_df = []
for theta in thetas:
    g.stack_to_gsd(name='/Rotations/rot_skel.gsd', crop=[L1,L3,L2,L4], rotate=theta)
    with Image.open(g.stack_dir+'/slice0.tiff') as im:
        img_crop = im.crop(box=[L2,L1,L4,L3])
        img_rot=im.rotate(theta*180/np.pi) #Rotate
        img_rot.save(g.dir+'/Rotations/slice0.tiff') 

    g.potential_distribution(0, [0,20], [727-20,727], R_j=0, rho_dim=1, F_dim=1)
    base.Node_labelling(g, g.P, 'P', 'test.gsd')
    #Ax, Ay = base.gyration_moments(G, sampling = 0.05)
    #L = g.L
    #Q = np.linalg.pinv(L)
    #O_eff = Q[-1,-1]+Q[-2,-2]-2*Q[-1,-2]
    
    #stop
    
    #O_eff_df.append(O_eff)
    #Ax_df.append(Ax)
    #Ay_df.append(Ay)
    #theta_df.append(theta)

#df_cont = pd.DataFrame(columns=['Theta','O_eff','Ax','Ay'])
#df_cont['O_eff'] = O_eff_df
#df_cont['Ax'] = Ax_df
#df_cont['Ay'] = Ay_df
#df_cont['Theta'] = theta_df
#df_cont.to_csv('AgNWN.csv')

#df_cont

In [ ]:
img_bin.shape
L2-L4

In [ ]:
img_slice = cv.imread(g.stack_dir+'/slice'+str(0)+'.tiff',cv.IMREAD_GRAYSCALE)
img_bin=[]
img_bin.append(img_slice)
img_bin = np.asarray(img_bin)
img_bin = np.swapaxes(img_bin, 0, 2)
print(img_bin.shape)
img_bin = img_bin[crop[2]:crop[3],crop[0]:crop[1]]
print(img_bin.shape)

In [ ]:
def timer(_class,method):
    """Print the runtime of the decorated function"""
    @functools.wraps(method)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = _method(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {method.__name__!r} in {run_time:.4f} secs")
        print(_class)
        print(method)
        return value
    return wrapper_timer